In [1]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 8.2 MB/s 
     |████████████████████████████████| 1.6 MB 55.8 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict,cross_val_score

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score, recall_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from kerastuner import HyperModel
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

from tensorflow.keras.callbacks import EarlyStopping
from keras import regularizers
from tensorflow.keras.layers import BatchNormalization

from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Embedding, Add

from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,GlobalMaxPooling1D
from keras.models import Sequential, Model
from keras.preprocessing import image
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.backend import clear_session
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Read training dataset

In [ ]:
#read train dataset
train=  pd.read_csv('/content/drive/MyDrive/Myocardial Infarction Detection/ptbdb/train.csv',encoding='utf-8')
test= pd.read_csv('/content/drive/MyDrive/Myocardial Infarction Detection/ptbdb/test.csv',encoding='utf-8')

## convert label to categorical

In [ ]:
from keras.utils.np_utils import to_categorical
X_train, y_train = train.iloc[: , :-1], train.iloc[: , -1]
X_test, y_test = test.iloc[: , :-1], test.iloc[: , -1]
y_train_categorical = to_categorical(y_train)
y_test_categorical=to_categorical(y_test)

In [ ]:
print("X shape=" +str(X_train.shape))
print("y shape=" +str(y_train.shape))


print("testX shape=" +str(X_test.shape))
print("testy shape=" +str(y_test.shape))

## Define some of function that are used as matixs 

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Optimize and build mode 

In [ ]:
from keras.models import load_model
model_CNN1= load_model('/content/drive/MyDrive/Myocardial Infarction Detection/ptbdb/model/CNN_Layer1.h5', custom_objects={'f1_m': f1_m,  'precision_m': precision_m, 'recall_m': recall_m} )
model_CNN2= load_model('/content/drive/MyDrive/Myocardial Infarction Detection/ptbdb/model/CNN_Layer2.h5', custom_objects={'f1_m': f1_m,  'precision_m': precision_m, 'recall_m': recall_m} )
model_CNN3= load_model('/content/drive/MyDrive/Myocardial Infarction Detection/ptbdb/model/CNN_Layer3.h5', custom_objects={'f1_m': f1_m,  'precision_m': precision_m, 'recall_m': recall_m} )


In [ ]:
members={'CNN1':model_CNN1,'CNN2':model_CNN2,'CNN3':model_CNN3}

In [ ]:
for key in members:
    for i in range(0, len(members[key].layers)-1):
        members[key].layers[i].trainable = False

In [ ]:
from numpy import dstack

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members,padded_train, padded_test, y_train,stack_Train, stack_Test):
    epoch=20
    BATCH_SIZE=1000
    for key in members:     
        
        if key=='CNN1':
            members['CNN1'].fit(padded_train, y_train, epochs=epoch, batch_size=BATCH_SIZE)
            yhat1 = members['CNN1'].predict(padded_train, verbose=0)
            yhat2 = members['CNN1'].predict(padded_test, verbose=0)
            
       
        elif key=='CNN2':

            members['CNN2'].fit(padded_train, y_train, epochs=epoch, batch_size=BATCH_SIZE)
            yhat1 = members['CNN2'].predict(padded_train,verbose=0)
            yhat2 = members['CNN2'].predict(padded_test,verbose=0)
            
        elif key=='CNN3':

            members['CNN3'].fit(padded_train, y_train, epochs=epoch, batch_size=BATCH_SIZE)
            yhat1 = members['CNN3'].predict(padded_train,verbose=0)
            yhat2 = members['CNN3'].predict(padded_test,verbose=0)


        if stack_Train is None:
            stack_Train = yhat1
        else:
            stack_Train = dstack((stack_Train, yhat1))
        
        if stack_Test is None:
            stack_Test = yhat2
        else:
            stack_Test = dstack((stack_Test, yhat2))
            
    stack_Train = stack_Train.reshape((stack_Train.shape[0], stack_Train.shape[1]*stack_Train.shape[2]))
    stack_Test = stack_Test.reshape((stack_Test.shape[0], stack_Test.shape[1]*stack_Test.shape[2]))
    return stack_Train, stack_Test



In [ ]:
SVM_DIC={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[]}
LR_DIC={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[]}
RF_DIC={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[]}

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average='macro '),
           'recall' : make_scorer(recall_score, average='macro '), 
           'f1_score' : make_scorer(f1_score, average='macro ')}

SFold = StratifiedKFold(n_splits=10)
stackX_Test=None
stackX_Train=None
stackX_Train,stackX_Test =stacked_dataset(members,X_train,X_test, y_train_categorical, stackX_Train, stackX_Test)

## Evaluating model

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
grid_search = GridSearchCV(estimator = SVC(), param_grid = param_grid, cv = SFold) 
grid_search.fit(stackX_Train, y_train)
model_SVM = grid_search.best_estimator_     

#scores_Train_SVM = cross_validate(model_SVM, stackX_Train, y_train, scoring=scoring,  cv=SFold)

yhat_SVM=model_SVM.predict(stackX_Test)



In [ ]:

print(grid_search.best_params_)

In [ ]:

SVM_DIC['Accurcy'].append(round(100*accuracy_score(y_test, yhat_SVM), 2))
SVM_DIC['Precision'].append(round(100*precision_score(y_test, yhat_SVM,average='macro'), 2))
SVM_DIC['Recall'].append(round(100*recall_score(y_test, yhat_SVM, average='macro'), 2))
SVM_DIC['F1'].append(round(100*f1_score(y_test, yhat_SVM, average='macro') , 2))     

print("SVM")
print(classification_report(y_test, yhat_SVM, target_names=['Normal','Unnormal'])) 
print("SVM Test")
ReultofTest=pd.DataFrame([])
ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(SVM_DIC['Accurcy']),2),
                                'PrecisionTest':round(np.mean(SVM_DIC['Precision']),2),
             'RecallTest' : round(np.mean(SVM_DIC['Recall']),2),'F1Test':round(np.mean(SVM_DIC['F1']),2)}, ignore_index=True)

ReultofTest.reindex(['AccuracyTest','PrecisionTest','RecallTest','F1Test'], axis=1)

In [ ]:

param_grid={"C":np.logspace(-3,3,7), "penalty":["l2"], "solver":['liblinear','newton-cg']}

grid_search_LR = GridSearchCV(estimator = LogisticRegression(), param_grid = param_grid, cv = SFold ) 
grid_search_LR.fit(stackX_Train, y_train)



In [ ]:
model_LR = grid_search_LR.best_estimator_
#scores_Train_LR = cross_validate(model_LR, stackX_Train, y_train, scoring=scoring,  cv=SFold)
yhat_LR=model_LR.predict(stackX_Test)

LR_DIC['Accurcy'].append(round(100*accuracy_score(y_test, yhat_LR), 2))
LR_DIC['Precision'].append(round(100*precision_score(y_test, yhat_LR,average='macro'), 2))
LR_DIC['Recall'].append(round(100*recall_score(y_test, yhat_LR, average='macro'), 2))
LR_DIC['F1'].append(round(100*f1_score(y_test, yhat_LR, average='macro') , 2))  


print("LR")
print(classification_report(y_test, yhat_LR, target_names=['Normal','Unnormal'])) 
print("logistic regression Test")
ReultofTest=pd.DataFrame([])
ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(LR_DIC['Accurcy']),2),
                                'PrecisionTest':round(np.mean(LR_DIC['Precision']),2),
             'RecallTest' : round(np.mean(LR_DIC['Recall']),2),'F1Test':round(np.mean(LR_DIC['F1']),2)}, ignore_index=True)




In [ ]:
print(grid_search_LR.best_params_)

In [ ]:
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [50, 60]


# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid

param_grid = {
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
grid_search_RF= GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, cv = SFold)

grid_search_RF.fit(stackX_Train, y_train)

In [ ]:
model_RF = grid_search_RF.best_estimator_   
yhat_RF=model_RF.predict(stackX_Test)
  
RF_DIC['Accurcy'].append(round(100*accuracy_score(y_test, yhat_RF), 2))
RF_DIC['Precision'].append(round(100*precision_score(y_test, yhat_RF,average='macro'), 2))
RF_DIC['Recall'].append(round(100*recall_score(y_test, yhat_RF, average='macro'), 2))
RF_DIC['F1'].append(round(100*f1_score(y_test, yhat_RF, average='macro') , 2))  

print("RF")
print(classification_report(y_test, yhat_RF, target_names=['Normal','Artial Premature'])) 

In [ ]:
print(grid_search_RF.best_params_)

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
  
grid_search_KNN= GridSearchCV(estimator =KNeighborsClassifier(), param_grid = param_grid, cv = SFold)

grid_search_KNN.fit(stackX_Train, y_train)

In [ ]:
KNN_DIC={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[]}
model_KNN = grid_search_KNN.best_estimator_   
yhat_KNN=model_KNN.predict(stackX_Test)
  
KNN_DIC['Accurcy'].append(round(100*accuracy_score(y_test, yhat_KNN), 2))
KNN_DIC['Precision'].append(round(100*precision_score(y_test, yhat_KNN,average='macro'), 2))
KNN_DIC['Recall'].append(round(100*recall_score(y_test, yhat_KNN, average='macro'), 2))
KNN_DIC['F1'].append(round(100*f1_score(y_test, yhat_KNN, average='macro') , 2))  

print("KNN")
print(classification_report(y_test, yhat_KNN, target_names=['Normal','Artial Premature'])) 

In [ ]:
print(grid_search_KNN.best_params_)